## Project: Personal Chef

"""

1. Take the left over ingredients left over in the kitchen.
2. Searches the web for recipes that use those ingredients and return them.
3. And can answer any follow up questions.

"""

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.jpg', multiple=False)
display(uploader)

FileUpload(value=(), accept='.jpg', description='Upload')

In [3]:
print(uploader.value)

({'name': 'IMG_0916.jpg', 'type': 'image/jpeg', 'size': 1489060, 'content': <memory at 0x105aa9480>, 'last_modified': datetime.datetime(2025, 12, 21, 20, 6, 15, 754000, tzinfo=datetime.timezone.utc)},)


In [4]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [5]:
from tavily import TavilyClient
from typing import Dict, Any
from langchain.tools import tool
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """ Performs web search on ingredients from image """
    
    tavily_client.search(query)

In [6]:
from langchain.messages import HumanMessage

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model = "gpt-5-nano",
    system_prompt = "Searches the web for recipes that use those ingredients and return them. Also, please be prepared to answer any follow up questions.",
    checkpointer = InMemorySaver(),
    tools = [web_search]
)

config = {"configurable": {"thread_id": "1"}}


multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me how can I use the ingredients present in this image, meaning for which recipes."},
    {"type": "image", "base64": img_b64, "mime_type": "image/jpg"}
])

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

print(response['messages'][-1].content)

Nice—looks like you’ve got eggs and probably almonds and milk in the fridge. Here are practical ways to use those ingredients right away, plus a few recipe ideas you can try. If you want, I can fetch step-by-step recipes with links afterward.

Recipe ideas using eggs, almonds, and milk (based on what I can see in the image)
- Almond French Toast
  - What you need: eggs, milk, bread, almonds (sliced or chopped), vanilla or cinnamon (optional)
  - How: whisk eggs with milk (and a splash of vanilla/cinnamon). Dip bread slices, cook on a hot greased skillet until golden. Finish with toasted almond slices on top and a drizzle of syrup if you like.
  - Why it’s good: uses the eggs and milk you have, plus almonds for a crunchy topping.

- Almond-Crusted Baked Eggs
  - What you need: eggs, milk (optional to mix in), almonds, a little oil or butter, salt/pepper
  - How: lightly toast chopped almonds in a pan. Whisk eggs with a splash of milk, season, pour into small ramekins, sprinkle toasted a

In [10]:
question = HumanMessage(content = "What is the Almond-Crusted Baked Eggs exactly? Is it to be included with boiled eggs or srambled eggs? Which one is your personal favorite?")
response = agent.invoke(
    {"messages": [question]},
    config
)
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content=[{'type': 'text', 'text': 'Tell me how can I use the ingredients present in this image, meaning for which recipes.'}, {'type': 'image', 'base64': '...', 'mime_type': 'image/jpg'}], additional_kwargs={}, response_metadata={}, id='98b98749-193b-4ebe-9ad2-243276dc0465'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 978, 'prompt_tokens': 2354, 'total_tokens': 3332, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 896, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpK6NAg59zcSbXJLFuhQVNzhKLEVN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b428c-0f42-7cd1-8a3b-e37b88299b3d-0', tool_calls=[{'name': 'web_search', '